### --- Day 24: Crossed Wires ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/24

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 24);
var (signals, gates) = inputLines.SeparateBy(line => line is "").ToArray();
WriteLines(signals);
WriteLines(gates);

Loading puzzle file: Day24.txt
Total lines: 313
Max line length: 18

x00: 1
x01: 1
x02: 1
x03: 1
x04: 0
vgh OR dhk -> kfp
qpb OR tdt -> z45
njd XOR hwt -> z33
y38 AND x38 -> srk
y25 AND x25 -> sth


This one looks reasonably straightforward for part 1, albeit mechanical. We can model the And, Or, Xor functions, arrange them according to the gate specifications, and crank the handle...

In [4]:
// Wires can carry 0, 1 or no value at all

using Signal = bool?;
using Wire = string;

In [5]:
class Device
{
    public Device(string[] gateDefs)
    {
        _gateFuncs = new();

        foreach (var gateDef in gateDefs)
        {
            var (a, op, b, _, output) = gateDef.Split(' ').ToArray();

            var gateFunc = MakeGateFunc(a, op, b, output);
            _gateFuncs.TryAdd(a, []);
            _gateFuncs.TryAdd(b, []);
            _gateFuncs[a].Add(gateFunc);
            _gateFuncs[b].Add(gateFunc);
        }
    }

    public void Simulate(string[] signalDefs)
    {
        _signals = new();

        foreach (var signalDef in signalDefs)
        {
            var (input, signalStr) = signalDef.Split(": ");
            Signal signal = (signalStr == "1");
            SendSignal(input, signal);
        }
    }

    Action MakeGateFunc(Wire aWire, string op, Wire bWire, Wire outputWire)
    {
        return () =>
        {
            var aVal = _signals.GetValueOrDefault(aWire);
            var bVal = _signals.GetValueOrDefault(bWire);

            var outputVal = (aVal, op, bVal) switch {
                (not null, "OR", not null) => aVal | bVal,
                (not null, "AND", not null) => aVal & bVal,
                (not null, "XOR", not null) => aVal ^ bVal,
                _ => null
            };

            SendSignal(outputWire, outputVal);
        };
    }

    void SendSignal(Wire inputWire, Signal signal)
    {
        if (signal is null) { return; }

        _signals[inputWire] = signal;
        foreach (var signalFunc in _gateFuncs.GetValueOrDefault(inputWire, []))
        {
            signalFunc();
        }
    }

    public long DecimalValue
    {
        get
        {
            var zOutputs = _signals.Where(kv => kv.Key[0] is 'z').OrderBy(kv => kv.Key).Reverse();

            return zOutputs.Aggregate(0L, (v, next) => (v <<= 1) | (next.Value.Value ? 1L : 0L));
        }
    }

    Dictionary<Wire, Signal> _signals;
    Dictionary<Wire, IList<Action>> _gateFuncs;
}

In [6]:
Device part1Device = new(gates);
part1Device.Simulate(signals);
var part1Answer = part1Device.DecimalValue;
Console.WriteLine(part1Answer);

45923082839246


In [7]:
// 45923082839246 is correct!
Ensure(45923082839246L, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/24

Ok, for part 2, the output wires of four pairs of gates have been swapped. It actually took me a minute or two to grasp what's going on here, as each line of gate definition contains IDs for its inputs as well as outputs. I found the easiest way to think about the problem is in pairs of lines. That is, we find two lines from our gate definitions, eg:

```
x00 AND y00 -> z05  <--- first
x01 AND y01 -> z02
x02 AND y02 -> z01  <--- second
```

...and just swap the output IDs. Other gates that use those same IDs in their inputs do not change.

On to solutions - the most basic approach would be to try all combinations of line pairs. An unlikely approach, but let's confirm:

In [9]:
// Ok, the braindead approach would be to try all permutations of 8 items. Is that even feasible?

void CheckOutputSwaps(string[] gateDefs)
{
    var totalGates = gateDefs.Count();
    Console.WriteLine($"Total gates: {totalGates}");

    var totalPairs = totalGates * (totalGates - 1);
    Console.WriteLine($"Total gate pairs: {totalPairs}");

    var combos = Enumerable.Range(0, 4).Select(i => totalPairs - i).Aggregate(1L, (a, b) => a * b);
    Console.WriteLine($"Total 4-combinations of pairs: {combos}");
}

CheckOutputSwaps(gates);

Total gates: 222
Total gate pairs: 49062
Total 4-combinations of pairs: 5793324824960810280


Ok, clearly way too many pair combinations to brute force. Perhaps we could reduce the number of gates involved, but we'll try a few other things first.

Let's try rendering as a flowchart - that worked for the counter puzzle last year:

In [10]:
async Task MakeItFlow(string[] gateDefs)
{
    StringBuilder sb = new();
    sb.AppendLine("flowchart LR");

    foreach (var line in gateDefs)
    {
        var (a, op, b, _, output) = line.Split(' ').ToArray();

        sb.AppendLine($"    {a} --> {output}");
        sb.AppendLine($"    {b} --> {output}");
    }

    await RunCellFromString(sb.ToString(), "mermaid");
}

await MakeItFlow(gates);

Seems the chart doesn't even load :) Pasting the text into the [Mermaid Chart Online Playground](https://www.mermaidchart.com) does however. We get the following image:

![Online chart rendered output](Day24.mermaid.svg)

Looks like it's semi-structured, but nothing immediately obvious jumps out.

I can only presume that, given we're dealing with bitwise addition and logic gates, the gates are structured as adders. Time to refresh my memory on adder gate structure. Thanks to [this article on VerilogCode.com](http://www.verilogcode.com/2015/05/digital-logic-design-full-adder-using.html) for the following images.

A *half adder* is structured like so:

![Half adder](https://lh5.googleusercontent.com/LGpTC5kvu-DDKV1zA5_3kuW0EDxgx8c1a32gldLwhff6XF14o67Q7WQ55ey2bzLCYEMj35gTfzpQTNJqfhc7B94iRQaCeGI3k3mROp1_yf2MFIPdgJDfhU6pbxy6KEGTNv2tXQU)

We should expect our first two inputs `x00` and `y00` to be a half adder, since there's no carry-in. Checking the gate definitions in the input text, it looks like this is the case!

```
y00 XOR x00 -> z00 --> Result
x00 AND y00 -> mjh --> Carry
```

Now, we should see the carry-out of this half adder passed into the *full adder* for bits `x01` and `y01`. A full adder looks like so:

![Full adder](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjCggOUxzlTPs-wHi0vUNmUCnvInN7KwvnA6vIRpWaGZcaOv0vZLIbPnLok-UKM5q1zNlduG0xMpDXvQY9u6LltgnxJB4Lpqi8x5RVKYOfmnuq4wX4HXiIUEcwdRlAqSmpqECdAiZzht5Y/s1600/Screen+Shot+2015-05-15+at+10.52.26+PM.png)

We know the first carry-in bit should be `mjh`, so let's check our gate definitions:

```
y01 XOR x01 -> rdj
rdj XOR mjh -> z01 --> Result

x01 AND y01 -> mkr
rdj AND mjh -> tqp
mkr OR tqp -> fhf  --> Cout
```

Again we have a match! So we know the gates are structured in adder formation. Also, we know that the first half-adder is correct, so we can focus on the full adders only: perhaps we can trace the gates, and throw errors when we fail validation. We might have to manually correct, but at the end we should be able to find our 4 swaps.

In [11]:
Wire AssertAdder(string[] gateDefs, Wire a, Wire b, Wire cin, Wire result)
{
    // Result gates
    var aXORb = Find(a, "XOR", b);
    var resultWire = Find(aXORb, "XOR", cin);
    
    if (resultWire != result)
    {
        throw new Exception($"Expected result wire to be {result}, was {resultWire}");
    }
    
    // Cout gates
    var aANDb = Find(a, "AND", b);
    var and2 = Find(aXORb, "AND", cin);
    var coutWire = Find(and2, "OR", aANDb);

    return coutWire;

    Wire Find(Wire a, Wire op, Wire b)
    {
        var targetLine1 = $"{a} {op} {b}";
        var targetLine2 = $"{b} {op} {a}";

        return gateDefs.SingleOrDefault(line => line.StartsWith(targetLine1) || line.StartsWith(targetLine2)) switch {
            var x when x is not null => x[^3..],
            _ => throw new Exception($"Cannot find {a} {op} {b}")
        };
    }
}

string CheckAllGates()
{
    // mjh is the first carry-in, as shown above
    var carryIn = "mjh";
    var gateDefs = gates.ToArray();

    // At each run, the verification would fail on the next crossed wire. The
    // following swaps are the result of manually inspecting and fixing each one.
    List<string> swaps = new();
    // Failure on x09
    swap("jnn XOR wpr -> rkf", "y09 AND x09 -> z09");
    // Failure on x20
    swap("stj XOR qkq -> jgb", "qkq AND stj -> z20");
    // Failure on x24
    swap("tnm XOR kkp -> vcg", "jnh OR njq -> z24");
    // Failure on x31
    swap("x31 XOR y31 -> rvc", "x31 AND y31 -> rrs");
    
    // Verify each adder, using the previous carry-out bit as the carry-in
    foreach (var i in Enumerable.Range(1, 44))
    {
        var ii = i.ToString("00");
        var a = $"x{ii}";
        var b = $"y{ii}";
        var result = $"z{ii}";

        // Console.WriteLine($"Checking {a} + {b} = {result}");
        var carryOut = AssertAdder(gateDefs, a, b, carryIn, result);
        // Console.WriteLine($"Passed. CarryOut is {carryOut}");
        carryIn = carryOut;
    }

    void swap(string a, string b)
    {
        var (aHead, aTail) = a.Split(" -> ");
        var (bHead, bTail) = b.Split(" -> ");

        gateDefs = gateDefs.Select(line => line
            .Replace(a, $"{aHead} -> {bTail}")
            .Replace(b, $"{bHead} -> {aTail}")
        ).ToArray();

        swaps.AddRange([aTail, bTail]);
    }

    swaps.Sort();
    return string.Join(",", swaps);
}

var part2Result = CheckAllGates();
Console.WriteLine(part2Result);

jgb,rkf,rrs,rvc,vcg,z09,z20,z24


In [12]:
// jgb,rkf,rrs,rvc,vcg,z09,z20,z24 is correct!
Ensure("jgb,rkf,rrs,rvc,vcg,z09,z20,z24", part2Result);